In [3]:
import os
from ultralytics import YOLO
import yaml
import matplotlib.pyplot as plt

# Paths
DATASET_PATH = r"E:\My Research Project\CODE\DataSampled"  
CUSTOM_MODEL_YAML = "yolov11.yaml"  
CHECKPOINT_PATH = "yolo_V11_results/yolo_V11_run/weights/last.pt"  
FINAL_MODEL_PATH = "yolo_V11_results/yolo_V11_run/weights/final.pt"  
BATCH_SIZE = 2  # Batch size

# --------------------------- Generate Dataset YAML ---------------------------
data_config = {
    "train": os.path.join(DATASET_PATH, "train"),  # Path to training images
    "val": os.path.join(DATASET_PATH, "val"),      # Path to validation images
    "nc": 99,                                      # Number of classes
    "names": [f"{i}" for i in range(99)]     # Dummy class names
}

# Save the dataset configuration as a YAML file
dataset_yaml_path = "data_config.yaml"
with open(dataset_yaml_path, "w") as file:
    yaml.dump(data_config, file, default_flow_style=False)
print(f"Dataset YAML saved at {dataset_yaml_path}")

# --------------------------- Load YOLO Model ---------------------------
# Load the YOLO model with the custom YAML file
model = YOLO(CUSTOM_MODEL_YAML, task="detect")

# --------------------------- Training with Resume ---------------------------
if os.path.exists(CHECKPOINT_PATH):
    print(f"Resuming training from checkpoint: {CHECKPOINT_PATH}")
    model.load(CHECKPOINT_PATH)  # Load the last checkpoint
else:
    print("Checkpoint not found! Starting fresh training.")


Dataset YAML saved at data_config.yaml
WARNING  no model scale passed. Assuming scale='n'.
Layer 0: Module = <class 'ultralytics.nn.modules.conv.Conv'>, Input Channels = 3, Output Channels = 16, Arguments = [3, 16, 3, 2]
Layer 1: Module = <class 'ultralytics.nn.modules.conv.Conv'>, Input Channels = 16, Output Channels = 32, Arguments = [16, 32, 3, 2]
Layer 2: Module = <class 'ultralytics.nn.modules.block.C3k2'>, Input Channels = 32, Output Channels = 64, Arguments = [32, 64, 1, False, 0.25]
Layer 3: Module = <class 'ultralytics.nn.modules.conv.Conv'>, Input Channels = 64, Output Channels = 64, Arguments = [64, 64, 3, 2]
Layer 4: Module = <class 'ultralytics.nn.modules.block.C3k2'>, Input Channels = 64, Output Channels = 128, Arguments = [64, 128, 1, False, 0.25]
Layer 5: Module = <class 'ultralytics.nn.modules.conv.Conv'>, Input Channels = 128, Output Channels = 128, Arguments = [128, 128, 3, 2]
Layer 6: Module = <class 'ultralytics.nn.modules.block.C3k2'>, Input Channels = 128, Output

In [4]:

# Train the model and save checkpoints
results = model.train(
    data=dataset_yaml_path,  # Path to dataset YAML file
    epochs=10,                # Number of epochs
    imgsz=640,               # Input image size (640x640)
    batch=BATCH_SIZE,        # Batch size
    device=0,                # GPU device index
    name="yolo_esca_run",    # Experiment name
    project="yolo_esca_results",  # Directory to save results
    resume=os.path.exists(CHECKPOINT_PATH),  # Resume if checkpoint exists
    save_period=1,           # Save checkpoint after every epoch
    save=True                # Save best model checkpoint
)

# --------------------------- Evaluation ---------------------------
# Validate the model after training
metrics = model.val()
print(f"Validation Results: {metrics}")


New https://pypi.org/project/ultralytics/8.3.53 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.50  Python-3.11.10 torch-2.5.1 CUDA:0 (NVIDIA GeForce RTX 3050 Laptop GPU, 4096MiB)
engine\trainer: task=detect, mode=train, model=yolov11.yaml, data=data_config.yaml, epochs=10, time=None, patience=100, batch=2, imgsz=640, save=True, save_period=1, cache=False, device=0, workers=8, project=yolo_esca_results, name=yolo_esca_run3, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None

train: Scanning E:\My Research Project\CODE\DataSampled\train\labels.cache... 6053 images, 1454 backgrounds, 0 corrupt: 100%|██████████| 6053/6053 [00:00<?, ?it/s]
val: Scanning E:\My Research Project\CODE\DataSampled\val\labels.cache... 1601 images, 250 backgrounds, 0 corrupt: 100%|██████████| 1601/1601 [00:00<?, ?it/s]


Plotting labels to yolo_esca_results\yolo_esca_run3\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=9.7e-05, momentum=0.9) with parameter groups 81 weight(decay=0.0), 88 weight(decay=0.0005), 87 bias(decay=0.0)
TensorBoard: model graph visualization added 
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to yolo_esca_results\yolo_esca_run3
Starting training for 10 epochs...
Closing dataloader mosaic

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       1/10     0.533G      3.025      8.222      3.879          4        640: 100%|██████████| 3027/3027 [05:36<00:00,  9.00it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 401/401 [00:30<00:00, 12.96it/s]


                   all       1601       3775      0.182   0.000779   0.000159   5.39e-05

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

                   all       1601       3775      0.111   0.000652   2.33e-05   1.05e-05

10 epochs completed in 0.930 hours.
Optimizer stripped from yolo_esca_results\yolo_esca_run3\weights\last.pt, 5.6MB
Optimizer stripped from yolo_esca_results\yolo_esca_run3\weights\best.pt, 5.6MB

Validating yolo_esca_results\yolo_esca_run3\weights\best.pt...
Ultralytics 8.3.50  Python-3.11.10 torch-2.5.1 CUDA:0 (NVIDIA GeForce RTX 3050 Laptop GPU, 4096MiB)
YOLOv11 summary (fused): 238 layers, 2,645,907 parameters, 0 gradients, 6.6 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 401/401 [00:14<00:00, 26.77it/s]


                   all       1601       3775    0.00169    0.00559   0.000871   9.11e-05
                     0         18         20          0          0          0          0
                     1         37         51          0          0          0          0
                     2         49         50          0          0          0          0
                     3         32         37          0          0          0          0
                     4         30         51          0          0          0          0
                     5         50         50          0          0          0          0
                     6         23         50          0          0          0          0
                     7         50         50          0          0          0          0
                     8         13         50          0          0          0          0
                     9         32         37          0          0          0          0
                    1

val: Scanning E:\My Research Project\CODE\DataSampled\val\labels.cache... 1601 images, 250 backgrounds, 0 corrupt: 100%|██████████| 1601/1601 [00:00<?, ?it/s]


MemoryError: 